In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from groq import Groq
from src.utils import get_secret

In [4]:
api_keys = get_secret("GROQ_API_KEYS").split(';')
client = Groq(api_key=api_keys[2])

Returning secret from environment variable `GROQ_API_KEYS`=`gs...a9`


In [5]:
PROMPT_TMPL = """Please translate the following single choice question and the 4 answer options regarding regulations of Los Andes University to English:
PREGUNTA: {question}

{options}

Your answer should consist of only 5 lines.
The expected format of those lines is:
QUESTION: <translation of PREGUNTA to *English*>
OPTION A: <translation of Opción A. to *English*>
OPTION B: <translation of Opción B. to *English*>
OPTION C: <translation of Opción C. to *English*>
OPTION D: <translation of Opción D. to *English*>
"""

GROQ_MODEL = "llama-3.2-1b-preview"

# rec = "¿Qué requisito de grado puede variar según el programa específico de maestría?,El promedio acumulado mínimo,El trabajo de grado,La asistencia a clases,El requisito de inglés".split(",")
rec="¿Cuál es el nivel de promedio acumulado necesario para recibir el grado Cum Laude?,Tener promedio acumulado superior a 4.5,Estar en el 3% superior del promedio histórico,Tener promedio acumulado de 5.0,Promedio superior a 4.0"

rec_parts = rec.split(",")
question = rec_parts[0]
options = [f"Opción {let}. {ans}" for let, ans in zip("ABCD", rec_parts[1:5])]

prompt = PROMPT_TMPL.format(question=question, options="\n".join(options))

print(f"====BEGIN-PROMPT===\n{prompt}\n=====END-PROMPT======\n")

chat_completion = client.chat.completions.create(
                        messages=[{
                            "role": "user",
                            "content": prompt
                        }],
                        model=GROQ_MODEL,
                        temperature=0.0
                    )

completion = chat_completion.choices[0].message.content
print(f"====BEGIN-COMPLETION===\n{completion}\n=======END-COMPLETION=======")

====BEGIN-PROMPT===
Please translate the following single choice question and the 4 answer options regarding regulations of Los Andes University to English:
PREGUNTA: ¿Cuál es el nivel de promedio acumulado necesario para recibir el grado Cum Laude?

Opción A. Tener promedio acumulado superior a 4.5
Opción B. Estar en el 3% superior del promedio histórico
Opción C. Tener promedio acumulado de 5.0
Opción D. Promedio superior a 4.0

Your answer should consist of only 5 lines.
The expected format of those lines is:
QUESTION: <translation of PREGUNTA to *English*>
OPTION A: <translation of Opción A. to *English*>
OPTION B: <translation of Opción B. to *English*>
OPTION C: <translation of Opción C. to *English*>
OPTION D: <translation of Opción D. to *English*>

=====END-PROMPT======

====BEGIN-COMPLETION===
QUESTION: Nivel de promedio acumulado necesario para Cum Laude
Opción A: Nivel de promedio acumulado superior a 4.5
Opción B: Nivel de promedio histórico superior al 3%
Opción C: Nivel 

In [6]:
PROMPT_TMPL = """Please translate the following to English:
"{sentence}"
Please provide ONLY A SINGLE English translation"""

GROQ_MODEL = "llama-3.2-1b-preview"

# rec = "¿Qué requisito de grado puede variar según el programa específico de maestría?,El promedio acumulado mínimo,El trabajo de grado,La asistencia a clases,El requisito de inglés".split(",")
rec="¿Cuál es el nivel de promedio acumulado necesario para recibir el grado Cum Laude?,Tener promedio acumulado superior a 4.5,Estar en el 3% superior del promedio histórico,Tener promedio acumulado de 5.0,Promedio superior a 4.0"

rec_parts = rec.split(",")
question = rec_parts[0]
options = [f"Opción {let}. {ans}" for let, ans in zip("ABCD", rec_parts[1:5])]

for a_str in rec_parts:
    prompt = PROMPT_TMPL.format(sentence=a_str)

    print(f"====BEGIN-PROMPT===\n{prompt}\n=====END-PROMPT======")

    chat_completion = client.chat.completions.create(
                            messages=[{
                                "role": "user",
                                "content": prompt
                            }],
                            model=GROQ_MODEL,
                            temperature=0.0
                        )

    completion = chat_completion.choices[0].message.content
    print(f"====BEGIN-COMPLETION===\n{completion}\n=======END-COMPLETION=======\n")

====BEGIN-PROMPT===
Please translate the following to English:
"¿Cuál es el nivel de promedio acumulado necesario para recibir el grado Cum Laude?"
Please provide ONLY A SINGLE English translation
=====END-PROMPT======
====BEGIN-COMPLETION===
"¿Cuál es el nivel de promedio acumulado necesario para recibir el grado Cum Laude?" 

 translates to "What is the accumulated average required to receive the Cum Laude degree?"
=======END-COMPLETION=======

====BEGIN-PROMPT===
Please translate the following to English:
"Tener promedio acumulado superior a 4.5"
Please provide ONLY A SINGLE English translation
=====END-PROMPT======
====BEGIN-COMPLETION===
"Total acumulado superior a 4.5"
=======END-COMPLETION=======

====BEGIN-PROMPT===
Please translate the following to English:
"Estar en el 3% superior del promedio histórico"
Please provide ONLY A SINGLE English translation
=====END-PROMPT======
====BEGIN-COMPLETION===
"Estar en el 3% superior del promedio histórico" translates to "Being at the 3%

In [24]:
from pathlib import Path
import pandas as pd
import torch as pt
from torch import nn, Tensor
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForCausalLM

import src.utils as ut
import src.trainer as trn
import src.finetuning as ft

from src.utils import LLAMA_MODEL_ID, login_to_hf_hub, load_test_df


In [8]:
login_to_hf_hub()
TOKENIZER = AutoTokenizer.from_pretrained(LLAMA_MODEL_ID)

Returning secret from environment variable `HF_API_KEY`=`hf...Gk`
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/teo/.cache/huggingface/token
Login successful


In [9]:
import re
line = "## Article"
re.search(r"(Article|Art\.|Artículo) *([0-9]+)", line)

In [16]:
from src.txt_dataset import TokenizedTxtDataset

text_fpaths = [
    Path("../data/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.translated.txt"),
    Path("../data/reglamento-maestria-web-2024.translated.txt"),
]

train_ds = TokenizedTxtDataset(text_fpaths,
                block_size=256,
                stride=128,
                tokenizer=TOKENIZER,
                start_pct=0.0,
                end_pct=90.0
            )


valid_ds = TokenizedTxtDataset(text_fpaths,
                block_size=256,
                stride=128,
                tokenizer=TOKENIZER,
                start_pct=90.0,
                end_pct=100.0
            )

print("len(ds):", len(train_ds), "max_stride_mult:", train_ds.max_stride_mult)

../data/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.translated.txt               :  49,682 bytes,    689 lines
../data/reglamento-maestria-web-2024.translated.txt                                             :  54,466 bytes,    686 lines
concat_files_to_str: returning 108,502 characters, whole text at: tokenized_txt_dataset.concat.txt
TokenizedTxtDs.__init__: len(all_text)=108,502
n_toks_raw: 20674 start_idx: 0 end_idx: 18606
n_toks_sampled:  18,606  input_ids length (padded):  18,688 block_size: 256 n_blocks: 73
../data/Estatutos-Universidad-de-los-Andes-2020-ratificados-MEN-RQ.translated.txt               :  49,682 bytes,    689 lines
../data/reglamento-maestria-web-2024.translated.txt                                             :  54,466 bytes,    686 lines
concat_files_to_str: returning 108,502 characters, whole text at: tokenized_txt_dataset.concat.txt
TokenizedTxtDs.__init__: len(all_text)=108,502
n_toks_raw: 20674 start_idx: 18606 end_idx: 20674
n_toks_sampled:   2

In [28]:
train_dl = DataLoader(train_ds, batch_size=4, shuffle=False)
for batch in train_dl:
    break

batch

{'input_ids': tensor([[128000,    271,  23417,  ...,    279,  85597,    323],
         [ 53412,  71056,   8812,  ...,    409,   2537,   1628],
         [  5201,    315,   3885,  ...,  67613,    409,   2537],
         [   288,  49357,     25,  ...,     11,    719,   1070]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
model = ut.load_raw_model()


In [25]:
ft.freeze_and_install_lora(model, lora_rank=16)

Parámetros sin LoRA: 167,772,160 || Parámetros con LoRA: 3,407,872  || Porcentaje de parámetros con LoRA: 1.99%


In [20]:

DEVICE = ut.module_device(model)
print(f"DEVICE: {DEVICE}")
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 8
LEARNING_RATE = 2e-4

trainer = trn.Trainer(
    train_ds=train_ds,
    train_batch_size=TRAIN_BATCH_SIZE,
    valid_ds=valid_ds,
    valid_batch_size=VALID_BATCH_SIZE,
    lr=LEARNING_RATE,
    device=DEVICE
)

DEVICE: cuda

Initializing trainer: device: cuda - len(train_ds)=146, len(valid_ds)=18


In [ ]:
def pred_next_token_loss(model: nn.Module, batch: dict[str, Tensor]) -> Tensor:
    return model(input_ids=batch['input_ids'],
                 attention_mask=batch['attention_mask'],
                 labels=batch['input_ids']).loss